In [1]:
import os 
import sys
import time


sys.path.append(os.path.abspath('../..'))


# DQN

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import DQN
from environment.env import Grid
from utils.checkpoints import find_last_checkpoint

obstacles = [((0.14625, 0.3325), (0.565, 0.55625)), 
             ((0.52875, 0.5375), (0.7375, 0.84125)), 
             ((0.0, 0.00125), (0.01625, 0.99125)), 
             ((0.0075, 0.00125), (0.99875, 0.04)), 
             ((0.98875, 0.0075), (0.99875, 1.0)), 
             ((0.00125, 0.9825), (0.99875, 1.0))]

env = Grid(
    obstacles=obstacles, 
    shear_range=(-.2, .2),
    stretch_range=(.4,1),
    render_mode="human"
)

last_checkpoint = find_last_checkpoint("../../agents/pretrained/MDP/DQN_continous_1vbmjd2a")
model = DQN.load(f"../../agents/pretrained/MDP/DQN_continous_1vbmjd2a/{last_checkpoint}", env=env)


mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=20, deterministic=True)
print(mean_reward, std_reward)
env.close()

# PPO

In [2]:
run = "PPO_continous_" + "bvmzgz49"

In [6]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO
from environment.env import Grid
from utils.checkpoints import find_last_checkpoint

obstacles = [((0.14625, 0.3325), (0.565, 0.55625)), 
             ((0.52875, 0.5375), (0.7375, 0.84125)), 
             ((0.0, 0.00125), (0.01625, 0.99125)), 
             ((0.0075, 0.00125), (0.99875, 0.04)), 
             ((0.98875, 0.0075), (0.99875, 1.0)), 
             ((0.00125, 0.9825), (0.99875, 1.0))]

env = Grid(
    obstacles=obstacles, 
    shear_range=(-.2, .2),
    stretch_range=(.4,1),
    render_mode="human"
)

last_checkpoint = find_last_checkpoint(f"../../agents/pretrained/MDP/{run}")
model = PPO.load(f"../../agents/pretrained/MDP/{run}/{last_checkpoint}", env=env)


mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=20, deterministic=True)
print(mean_reward, std_reward)

env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-7.1875 6.574512814650223


In [8]:
def eval_agent_mdp(agent,env,num_episodes,max_episode_steps,render):
    """Returns
        - episode_transition: list of list of tuples (s,a,r,s',done), t[i] is the ith episode
        - beliefs: list of beliefs at each time step 
    """
    for i in range(num_episodes):
        s, _ = env.reset()

        totalReward = 0.0
        done = False
        steps = 0

        while not done and steps < max_episode_steps:

            best_action, _ = agent.predict(s,deterministic=True)
            next_state, reward, terminated, truncated, info = env.step(best_action)
            totalReward += reward            

            done = terminated or truncated
            if render:
                # print("State", s)
                # print("Action: ", best_action)
                # print("Reward:     " + str(totalReward) + "  ")
                # print("Next State: ", next_state)
                # print("\n")
                env.render()

            s = next_state
            steps += 1
            time.sleep(0.02)

env = Grid(
    render_mode="human"
)

eval_agent_mdp(model,env,10,100,True)

In [12]:
env = Grid(
    render_mode="human"
)
env.reset()
env.render()

(None, None, None, None, None)

In [13]:
env.reset(4256)
env.render()

(None, None, None, None, None)

In [14]:
env.close()

In [15]:
import numpy as np

np.random.seed(2)
np.random.randint(0, 1000)

168